# Navigation

This notebook was provided by Udacity as example.

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

[![](https://trello-attachments.s3.amazonaws.com/5d93d97e715aba7c97fde8f7/5e9e457961d16e8a352b67c9/da4327f262b337b51a85826ca5e5cf6f/image.png)](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893)

---

# Index

- [1. Setup the Environment](#1.-Setup-the-Environment)
- [2. Start the Environment](#2.-Start-the-Environment)
- [3. Examine the State and Action Spaces](#3.-Examine-the-State-and-Action-Spaces)
- [4. Develop the Agent to collect bananas](#4.-Develop-the-Agent-to-collect-bananas)
 - [4.1 Define trainer function](#4.1-Define-trainer-function)
 - [4.2 Training agent on environment](#4.2.-Training-agent-on-environment)
 - [4.3 Plot agent losses along the training session](#4.2-Plot-agent-losses-along-the-training-session)
 - [4.4 Save weights](#4.4-Save-weights)
 - [4.5 Load weights](#4.5-Load-weights)
- [5. Test Agent on Environment](#5.-Test-Agent-on-Environment)

### 1. Setup the Environment

<img align="left" width="150" src="https://www.nclouds.com/img/services/toolkit/sagemaker.png"/>

This notebook was developed on AWS SageMaker.

The kernel used is **conda_python3**

To setup this environment on SageMaker you need to run the next 2 cells.

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip -q install ./python

### 2. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt

from collections import deque

from agent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `env_path` variable to match the location of the Unity environment that you downloaded.

- **Linux** (x86_64): `"/Banana_Linux/Banana.x86_64"`
- **Linux** (x86_64, headless): `"/Banana_Linux_NoVis/Banana.x86_64"`

If your system isn't in the list, please, download the environment from here.

The `file_name` parameter needs to be something like that:

```
env = UnityEnvironment(file_name="Banana_Linux_NoVis/Banana.x86_64")
```

In [ ]:
env_path = "Banana_Linux_NoVis/Banana.x86_64"
#env_path = "Banana_Linux/Banana.x86_64"

env = UnityEnvironment(file_name=env_path)

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 3. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 4. Develop the Agent to collect bananas

In the next code cells, we will train the Agent to work on environment.

### 4.1 Define trainer function

In [ ]:
def dqn_trainer(agent, n_episodes=100, print_range=10, eps_start=1.0, eps_end=0.01, eps_decay=0.995, early_stop=13):
    """Deep Q-Learning trainer.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        print_range (int): range to print partials results
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
        early_stop (int): Stop training when achieve a defined score respecting 10 min n_episodes.
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=print_range)  # last 100 scores
    scores_mean = []
    eps = eps_start                    # initialize epsilon
    for i in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        
        while True:
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0] 
            agent.step(state, action, reward, next_state, done, i)
            state = next_state
            score += reward
            if done:
                break
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(scores_window)), end="")
        if i % print_range == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i, np.mean(scores_window)))
            
        if np.mean(scores_window) >= early_stop and i > 10:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i, np.mean(scores_window)))
            break
    return scores

### 4.2. Training agent on environment

In [ ]:
import importlib
import torch
import agent
importlib.reload(agent)

In [ ]:
%%time

agent = agent.Agent(state_size, action_size, seed=1412, nb_hidden=128, learning_rate=5.0e-4,
              memory_size=int(3e4), prioritized_memory=True, batch_size=64,
              gamma=0.99, tau=0.001, small_eps=1e-5, update_every=5)

scores = dqn_trainer(agent, n_episodes=100)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.show()

### 4.3 Plot agent losses along the training session

In [ ]:
plt.plot(agent.losses)
plt.title("Losses")
plt.show()

### 4.4 Save weights

In [ ]:
model_path = "checkpoint.pth"
agent.save_model(model_path)

### 4.5 Load weights

In [ ]:
model_path = "checkpoint.pth"
agent.load_model(model_path)

### 5. Test Agent on Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = agent.act(state, eps)
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if done:
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
if False:
    env.close()